In [1]:
from collections import defaultdict
import csv
from itertools import islice, chain, combinations, permutations
import itertools
from functools import reduce
import os
import sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, Image
import scipy
import seaborn as sns
import sklearn
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm_notebook
import torch
from torchvision import models, transforms, datasets

In [3]:
#NOTE: Votes are counted as [fake, real], not [incorrect, correct] !

dataset_counts = defaultdict(int)
img_counts = defaultdict(lambda : defaultdict(int))
unique_imgs = defaultdict(set)
img_votes = defaultdict(lambda: defaultdict(lambda: [0,0])) #no,yes tuples by dataset and image

with open('../all_gans_inf.csv') as f:
    reader = csv.DictReader(f)
    for row in islice(reader, None):
        img = row['img']        
#         dataset_name = img[:img.index('/')]
        split_name = img.split('/')
        assert(len(split_name) == 2)
        dataset_name, img_name = split_name
        
        vote_index = 1 if row['correctness'] == row['realness'] else 0
        if dataset_name == 'began5000' and row['realness'] == 'True': print(row)

        img_votes[dataset_name][img_name][vote_index] += 1
        
        dataset_counts[dataset_name] += 1
        img_counts[dataset_name][img_name] += 1
unique_imgs = {dataset: len(img_counts[dataset]) for dataset in img_counts}

In [4]:
gan_models = ['progan', 'stylegan', 'began', 'wgan']

progan_votes = img_votes['progan5000']
began_votes = img_votes['began5000']
stylegan_votes = img_votes['styleganceleba5000']
wgan_votes = img_votes['wgangp5000']
votes = {k:v for k,v in chain(progan_votes.items(), stylegan_votes.items(), began_votes.items(), wgan_votes.items())}
len(votes)

11984

In [6]:
progan_filenames = torch.load('./knn_experiment/progan_filenames.pt', map_location=torch.device('cpu'))
progan_filenames = [os.path.split(x)[-1] for x in progan_filenames]
progan_distance_features = torch.load('./knn_experiment/progan_distance_features.pt', map_location=torch.device('cpu'))

stylegan_filenames = torch.load('./knn_experiment/stylegan_filenames.pt', map_location=torch.device('cpu'))
stylegan_filenames = [os.path.split(x)[-1] for x in stylegan_filenames]
stylegan_distance_features = torch.load('./knn_experiment/stylegan_distance_features.pt', map_location=torch.device('cpu'))

began_filenames = torch.load('./knn_experiment/began_filenames.pt', map_location=torch.device('cpu'))
began_filenames = [os.path.split(x)[-1] for x in began_filenames]
began_distance_features = torch.load('./knn_experiment/began_distance_features.pt', map_location=torch.device('cpu'))

wgan_filenames = torch.load('./knn_experiment/wgan_filenames.pt', map_location=torch.device('cpu'))
wgan_filenames = [os.path.split(x)[-1] for x in wgan_filenames]
wgan_distance_features = torch.load('./knn_experiment/wgan_distance_features.pt', map_location=torch.device('cpu'))

filenames = {'progan' : progan_filenames,
        'stylegan' : stylegan_filenames,
        'began' : began_filenames,
        'wgan' : wgan_filenames}

distance_features = {'progan' : progan_distance_features,
        'stylegan' : stylegan_distance_features,
        'began' : began_distance_features,
        'wgan' : wgan_distance_features}

In [7]:
with open('progan_train_set.txt') as f:
    progan_train_files = [os.path.split(x.strip())[-1] for x in f.readlines()]
#print(progan_train_files[:5])

with open('progan_val_set.txt') as f:
    progan_val_files = [os.path.split(x.strip())[-1] for x in f.readlines()]
#print(progan_val_files[:5])

with open('stylegan_train_set.txt') as f:
    stylegan_train_files = [os.path.split(x.strip())[-1] for x in f.readlines()]
#print(stylegan_train_files[:5])

with open('stylegan_val_set.txt') as f:
    stylegan_val_files = [os.path.split(x.strip())[-1] for x in f.readlines()]
#print(stylegan_val_files[:5])

with open('began_train_set.txt') as f:
    began_train_files = [os.path.split(x.strip())[-1] for x in f.readlines()]
#print(began_train_files[:5])

with open('began_val_set.txt') as f:
    began_val_files = [os.path.split(x.strip())[-1] for x in f.readlines()]
#print(began_val_files[:5])

with open('wgan_train_set.txt') as f:
    wgan_train_files = [os.path.split(x.strip())[-1] for x in f.readlines()]
#print(began_train_files[:5])

with open('wgan_val_set.txt') as f:
    wgan_val_files = [os.path.split(x.strip())[-1] for x in f.readlines()]
#print(began_val_files[:5])



with open('progan_test_set.txt') as f:
    progan_test_files = [os.path.split(x.strip())[-1] for x in f.readlines()]

with open('stylegan_test_set.txt') as f:
    stylegan_test_files = [os.path.split(x.strip())[-1] for x in f.readlines()]

with open('began_test_set.txt') as f:
    began_test_files = [os.path.split(x.strip())[-1] for x in f.readlines()]

with open('wgan_test_set.txt') as f:
    wgan_test_files = [os.path.split(x.strip())[-1] for x in f.readlines()]

train_files = {'progan' : progan_train_files,
        'stylegan' : stylegan_train_files,
        'began' : began_train_files,
        'wgan' : wgan_train_files}

val_files = {'progan' : progan_val_files,
        'stylegan' : stylegan_val_files,
        'began' : began_val_files,
        'wgan' : wgan_val_files}

test_files = {'progan' : progan_test_files,
        'stylegan' : stylegan_test_files,
        'began' : began_test_files,
        'wgan' : wgan_test_files}

In [8]:
progan_features_by_file = {progan_filenames[i] : progan_distance_features[i] for i in range(len(progan_filenames))}
print(len(progan_features_by_file))

stylegan_features_by_file = {stylegan_filenames[i] : stylegan_distance_features[i] for i in range(len(stylegan_filenames))}
print(len(stylegan_features_by_file))

began_features_by_file = {began_filenames[i] : began_distance_features[i] for i in range(len(began_filenames))}
print(len(began_features_by_file))

wgan_features_by_file = {wgan_filenames[i] : wgan_distance_features[i] for i in range(len(wgan_filenames))}
print(len(wgan_features_by_file))

# features_by_file = {'progan' : progan_features_by_file,
#         'stylegan' : stylegan_features_by_file,
#         'began' : began_features_by_file,
#         'wgan' : wgan_features_by_file}

features_by_file = {k:v for k,v in chain(progan_features_by_file.items(), stylegan_features_by_file.items(), 
                    began_features_by_file.items(), wgan_features_by_file.items())}

print(len(features_by_file))

2233
3103
1966
4251
11553


In [10]:
dense_labels = {}
#dense_labels['val'] = {}
#dense_labels['test'] = {}

for m in gan_models:
    for val_or_test in ('val', 'test'):
        with open('dense_labels/{}_{}_dense_labels.csv'.format(m, val_or_test)) as f:
            reader = csv.reader(f)
            for row in islice(reader,None):
                dense_labels[row[0]] = round(float(row[1]))
                #print(row[0], float(row[1]))

len(dense_labels)

1592

In [43]:
x = []
y = []
all_train_files = progan_train_files + stylegan_train_files + began_train_files + wgan_train_files

for f in all_train_files:
    false_votes, real_votes = votes[f]
    for i in range(false_votes):
        x.append(features_by_file[f])
        y.append(0.)
    for i in range(real_votes):
        x.append(features_by_file[f])
        y.append(1.)
        
#x = np.array([features_by_file[f] for f in all_train_files])
#y = [votes[f] for f in all_train_files]
x = torch.tensor(x)
y = torch.tensor(y).reshape(-1,1)
x.shape, y.shape

(torch.Size([13686, 7]), torch.Size([13686, 1]))

In [70]:
n1 = len(progan_train_files)
n2 = n1 + len(stylegan_train_files)
n3 = n2 + len(began_train_files)

x_progan = x[:n1]
y_progan = y[:n1]

x_stylegan = x[n1:n2]
y_stylegan = y[n1:n2]

x_began = x[n2:n3]
y_began = y[n2:n3]

x_wgan = x[n3:]
y_wgan = y[n3:]

gan_xs = [x_progan, x_stylegan, x_began, x_wgan]
gan_ys = [y_progan, y_stylegan, y_began, y_wgan]
gan_models

['progan', 'stylegan', 'began', 'wgan']

In [89]:
#model = torch.nn.Sequential(
#          torch.nn.Linear(7, 100),
#          torch.nn.ReLU(),
#          torch.nn.Linear(100, 100),
#          torch.nn.ReLU(),
#          torch.nn.Linear(100, 1),
#        )

loss_fn = torch.nn.BCEWithLogitsLoss()
opt = torch.optim.Adam(model.parameters(), lr=3e-4)
N = x.size(0)
batch_size = 128
num_batches = N//batch_size if N%batch_size == 0 else N//batch_size + 1
#num_batches = 3


for epoch in range(10000):
    print('\n\n', epoch)
    indices = torch.randperm(N)
    
    for b in range(num_batches):
        batch_indices = indices[b*batch_size : (b+1) * batch_size]
        x_batch = x[batch_indices]
        y_batch = y[batch_indices]
        y_batch_pred = model(x_batch)
        loss = loss_fn(y_batch_pred, y_batch)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
    
    #accuracy = ((y_pred > 0.5).type(torch.FloatTensor) == y).type(torch.FloatTensor).mean()
        #print(accuracy)
        
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    y_val_pred = model(x_val)
    val_loss = loss_fn(y_val_pred, y_val)
    print('Train loss: {}, val loss: {}'.format(loss, val_loss))
    for i in range(4):
        print('Evaluating {}'.format(gan_models[i]))
        y_pred = model(gan_xs[i])
        print(y_pred.mean())
        accuracy = ((y_pred > 0.5).type(torch.FloatTensor) == gan_ys[i]).type(torch.FloatTensor).mean()
        print('Train accuracy: {}'.format(accuracy))
        
        y_pred = model(gan_val_xs[i])
        accuracy = ((y_pred > 0.5).type(torch.FloatTensor) == gan_val_ys[i]).type(torch.FloatTensor).mean()
        print('Val accuracy: {}'.format(accuracy))

        



 0
Train loss: 0.3428635001182556, val loss: 0.4837082326412201
Evaluating progan
tensor(-0.5610, grad_fn=<MeanBackward0>)
Train accuracy: 0.6675993204116821
Val accuracy: 0.5964125394821167
Evaluating stylegan
tensor(0.0257, grad_fn=<MeanBackward0>)
Train accuracy: 0.6399516463279724
Val accuracy: 0.5096774101257324
Evaluating began
tensor(0.3547, grad_fn=<MeanBackward0>)
Train accuracy: 0.6270648241043091
Val accuracy: 0.5255101919174194
Evaluating wgan
tensor(-3.9758, grad_fn=<MeanBackward0>)
Train accuracy: 0.9580464363098145
Val accuracy: 0.9271255135536194


 1
Train loss: 0.33956897258758545, val loss: 0.48487070202827454
Evaluating progan
tensor(-0.4360, grad_fn=<MeanBackward0>)
Train accuracy: 0.6810296773910522
Val accuracy: 0.6233183741569519
Evaluating stylegan
tensor(0.1489, grad_fn=<MeanBackward0>)
Train accuracy: 0.6580749154090881
Val accuracy: 0.5032258033752441
Evaluating began
tensor(0.4434, grad_fn=<MeanBackward0>)
Train accuracy: 0.664548933506012
Val accuracy: 0

Train loss: 0.342195600271225, val loss: 0.4849468767642975
Evaluating progan
tensor(-0.5787, grad_fn=<MeanBackward0>)
Train accuracy: 0.6703973412513733
Val accuracy: 0.605381190776825
Evaluating stylegan
tensor(0.0008, grad_fn=<MeanBackward0>)
Train accuracy: 0.6443818211555481
Val accuracy: 0.4935483932495117
Evaluating began
tensor(0.3077, grad_fn=<MeanBackward0>)
Train accuracy: 0.6410419344902039
Val accuracy: 0.5255101919174194
Evaluating wgan
tensor(-4.0864, grad_fn=<MeanBackward0>)
Train accuracy: 0.9602142572402954
Val accuracy: 0.9382591247558594


 16
Train loss: 0.34040313959121704, val loss: 0.48078852891921997
Evaluating progan
tensor(-0.6215, grad_fn=<MeanBackward0>)
Train accuracy: 0.6703973412513733
Val accuracy: 0.6008968353271484
Evaluating stylegan
tensor(0.0167, grad_fn=<MeanBackward0>)
Train accuracy: 0.6480064392089844
Val accuracy: 0.48064514994621277
Evaluating began
tensor(0.2809, grad_fn=<MeanBackward0>)
Train accuracy: 0.6365946531295776
Val accuracy: 0.515

Train loss: 0.344389945268631, val loss: 0.48973503708839417
Evaluating progan
tensor(-0.4031, grad_fn=<MeanBackward0>)
Train accuracy: 0.6855064630508423
Val accuracy: 0.5829596519470215
Evaluating stylegan
tensor(0.1784, grad_fn=<MeanBackward0>)
Train accuracy: 0.648811936378479
Val accuracy: 0.4838709533214569
Evaluating began
tensor(0.4078, grad_fn=<MeanBackward0>)
Train accuracy: 0.6391359567642212
Val accuracy: 0.5204081535339355
Evaluating wgan
tensor(-3.6408, grad_fn=<MeanBackward0>)
Train accuracy: 0.9570262432098389
Val accuracy: 0.9352226853370667


 31
Train loss: 0.35170143842697144, val loss: 0.48341211676597595
Evaluating progan
tensor(-0.7899, grad_fn=<MeanBackward0>)
Train accuracy: 0.6474538445472717
Val accuracy: 0.6322869658470154
Evaluating stylegan
tensor(-0.1818, grad_fn=<MeanBackward0>)
Train accuracy: 0.6258558034896851
Val accuracy: 0.4903225898742676
Evaluating began
tensor(0.1291, grad_fn=<MeanBackward0>)
Train accuracy: 0.5946632623672485
Val accuracy: 0.48

Train loss: 0.3515223264694214, val loss: 0.489990234375
Evaluating progan
tensor(-0.7912, grad_fn=<MeanBackward0>)
Train accuracy: 0.6233911514282227
Val accuracy: 0.5829596519470215
Evaluating stylegan
tensor(-0.2184, grad_fn=<MeanBackward0>)
Train accuracy: 0.5976641178131104
Val accuracy: 0.46451613306999207
Evaluating began
tensor(0.0923, grad_fn=<MeanBackward0>)
Train accuracy: 0.5832274556159973
Val accuracy: 0.5
Evaluating wgan
tensor(-4.1841, grad_fn=<MeanBackward0>)
Train accuracy: 0.9589390754699707
Val accuracy: 0.9331983923912048


 46
Train loss: 0.33899036049842834, val loss: 0.4858262538909912
Evaluating progan
tensor(-0.4549, grad_fn=<MeanBackward0>)
Train accuracy: 0.6810296773910522
Val accuracy: 0.591928243637085
Evaluating stylegan
tensor(0.1372, grad_fn=<MeanBackward0>)
Train accuracy: 0.6552557349205017
Val accuracy: 0.5
Evaluating began
tensor(0.4152, grad_fn=<MeanBackward0>)
Train accuracy: 0.6493011713027954
Val accuracy: 0.5408163070678711
Evaluating wgan
ten

Train loss: 0.3428388237953186, val loss: 0.4759068489074707
Evaluating progan
tensor(-0.3546, grad_fn=<MeanBackward0>)
Train accuracy: 0.6944599747657776
Val accuracy: 0.6098654866218567
Evaluating stylegan
tensor(0.2096, grad_fn=<MeanBackward0>)
Train accuracy: 0.6592831015586853
Val accuracy: 0.5064516067504883
Evaluating began
tensor(0.4696, grad_fn=<MeanBackward0>)
Train accuracy: 0.6613723039627075
Val accuracy: 0.5204081535339355
Evaluating wgan
tensor(-3.7875, grad_fn=<MeanBackward0>)
Train accuracy: 0.9575363397598267
Val accuracy: 0.9321862459182739


 61
Train loss: 0.35157766938209534, val loss: 0.49721503257751465
Evaluating progan
tensor(-0.3846, grad_fn=<MeanBackward0>)
Train accuracy: 0.701734721660614
Val accuracy: 0.6008968353271484
Evaluating stylegan
tensor(0.1278, grad_fn=<MeanBackward0>)
Train accuracy: 0.6584776639938354
Val accuracy: 0.5
Evaluating began
tensor(0.3916, grad_fn=<MeanBackward0>)
Train accuracy: 0.6473951935768127
Val accuracy: 0.5051020383834839
E

...

Train loss: 0.36470550298690796, val loss: 0.5117349028587341
Evaluating progan
tensor(-0.8451, grad_fn=<MeanBackward0>)
Train accuracy: 0.6362618803977966
Val accuracy: 0.5784753561019897
Evaluating stylegan
tensor(-0.1395, grad_fn=<MeanBackward0>)
Train accuracy: 0.6238421201705933
Val accuracy: 0.5
Evaluating began
tensor(0.2223, grad_fn=<MeanBackward0>)
Train accuracy: 0.6175349354743958
Val accuracy: 0.4897959232330322
Evaluating wgan
tensor(-4.5037, grad_fn=<MeanBackward0>)
Train accuracy: 0.9575363397598267
Val accuracy: 0.9352226853370667


 1606
Train loss: 0.34051281213760376, val loss: 0.4907752573490143
Evaluating progan
tensor(-0.7838, grad_fn=<MeanBackward0>)
Train accuracy: 0.6485730409622192
Val accuracy: 0.605381190776825
Evaluating stylegan
tensor(-0.1253, grad_fn=<MeanBackward0>)
Train accuracy: 0.6298832297325134
Val accuracy: 0.4741935431957245
Evaluating began
tensor(0.1760, grad_fn=<MeanBackward0>)
Train accuracy: 0.6181702613830566
Val accuracy: 0.48469388484954

Train loss: 0.34052008390426636, val loss: 0.49725013971328735
Evaluating progan
tensor(-0.4645, grad_fn=<MeanBackward0>)
Train accuracy: 0.6843872666358948
Val accuracy: 0.5964125394821167
Evaluating stylegan
tensor(0.0828, grad_fn=<MeanBackward0>)
Train accuracy: 0.6492146849632263
Val accuracy: 0.5032258033752441
Evaluating began
tensor(0.3197, grad_fn=<MeanBackward0>)
Train accuracy: 0.6543837189674377
Val accuracy: 0.47959184646606445
Evaluating wgan
tensor(-3.8264, grad_fn=<MeanBackward0>)
Train accuracy: 0.9580464363098145
Val accuracy: 0.9342105388641357


 1621
Train loss: 0.33741530776023865, val loss: 0.4820699095726013
Evaluating progan
tensor(-0.4717, grad_fn=<MeanBackward0>)
Train accuracy: 0.6832680702209473
Val accuracy: 0.5695067048072815
Evaluating stylegan
tensor(0.1327, grad_fn=<MeanBackward0>)
Train accuracy: 0.6500201225280762
Val accuracy: 0.49677419662475586
Evaluating began
tensor(0.4128, grad_fn=<MeanBackward0>)
Train accuracy: 0.6391359567642212
Val accuracy:

Train loss: 0.34469911456108093, val loss: 0.49762168526649475
Evaluating progan
tensor(-0.6389, grad_fn=<MeanBackward0>)
Train accuracy: 0.6754336953163147
Val accuracy: 0.6098654866218567
Evaluating stylegan
tensor(-0.0363, grad_fn=<MeanBackward0>)
Train accuracy: 0.6415626406669617
Val accuracy: 0.5161290168762207
Evaluating began
tensor(0.1754, grad_fn=<MeanBackward0>)
Train accuracy: 0.621982216835022
Val accuracy: 0.5153061151504517
Evaluating wgan
tensor(-3.8301, grad_fn=<MeanBackward0>)
Train accuracy: 0.9566437005996704
Val accuracy: 0.9352226853370667


 1636
Train loss: 0.33714520931243896, val loss: 0.48531630635261536
Evaluating progan
tensor(-0.5664, grad_fn=<MeanBackward0>)
Train accuracy: 0.6731953024864197
Val accuracy: 0.6143497824668884
Evaluating stylegan
tensor(0.0326, grad_fn=<MeanBackward0>)
Train accuracy: 0.6403543949127197
Val accuracy: 0.5
Evaluating began
tensor(0.2591, grad_fn=<MeanBackward0>)
Train accuracy: 0.6321473717689514
Val accuracy: 0.5051020383834

Train loss: 0.35436543822288513, val loss: 0.506952166557312
Evaluating progan
tensor(-0.5789, grad_fn=<MeanBackward0>)
Train accuracy: 0.6759932637214661
Val accuracy: 0.5829596519470215
Evaluating stylegan
tensor(-0.0101, grad_fn=<MeanBackward0>)
Train accuracy: 0.6322996616363525
Val accuracy: 0.4838709533214569
Evaluating began
tensor(0.3651, grad_fn=<MeanBackward0>)
Train accuracy: 0.6391359567642212
Val accuracy: 0.5051020383834839
Evaluating wgan
tensor(-4.3427, grad_fn=<MeanBackward0>)
Train accuracy: 0.9589390754699707
Val accuracy: 0.9331983923912048


 1651
Train loss: 0.34061160683631897, val loss: 0.49617084860801697
Evaluating progan
tensor(-0.8345, grad_fn=<MeanBackward0>)
Train accuracy: 0.6536093950271606
Val accuracy: 0.5650224089622498
Evaluating stylegan
tensor(-0.2364, grad_fn=<MeanBackward0>)
Train accuracy: 0.6194120049476624
Val accuracy: 0.4838709533214569
Evaluating began
tensor(0.0298, grad_fn=<MeanBackward0>)
Train accuracy: 0.6099110841751099
Val accuracy: 

KeyboardInterrupt: 

In [87]:
x_val = []
y_val = []
all_val_files = progan_val_files + stylegan_val_files + began_val_files + wgan_val_files

for f in all_val_files:
    false_votes, real_votes = votes[f]
    for i in range(false_votes):
        x_val.append(features_by_file[f])
        y_val.append(0.)
    for i in range(real_votes):
        x_val.append(features_by_file[f])
        y_val.append(1.)
        
#x = np.array([features_by_file[f] for f in all_train_files])
#y = [votes[f] for f in all_train_files]
x_val = torch.tensor(x_val)
y_val = torch.tensor(y_val).reshape(-1,1)
x_val.shape, y_val.shape


n1 = len(progan_val_files)
n2 = n1 + len(stylegan_val_files)
n3 = n2 + len(began_val_files)

x_val_progan = x_val[:n1]
y_val_progan = y_val[:n1]

x_val_stylegan = x_val[n1:n2]
y_val_stylegan = y_val[n1:n2]

x_val_began = x_val[n2:n3]
y_val_began = y_val[n2:n3]

x_val_wgan = x_val[n3:]
y_val_wgan = y_val[n3:]

gan_val_xs = [x_val_progan, x_val_stylegan, x_val_began, x_val_wgan]
gan_val_ys = [y_val_progan, y_val_stylegan, y_val_began, y_val_wgan]
[len(x) for x in gan_val_ys], n1, n2, n3, y_val.shape

([223, 310, 196, 988], 223, 533, 729, torch.Size([1717, 1]))

In [88]:



print(loss)
for i in range(4):
    print('Evaluating {}'.format(gan_models[i]))
    y_pred = model(gan_val_xs[i])
    print(y_pred.mean())
    accuracy = ((y_pred > 0.5).type(torch.FloatTensor) == gan_val_ys[i]).type(torch.FloatTensor).mean()
    print(accuracy)

tensor(0.5012, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Evaluating progan
tensor(-0.9498, grad_fn=<MeanBackward0>)
tensor(0.6099)
Evaluating stylegan
tensor(-0.1987, grad_fn=<MeanBackward0>)
tensor(0.5000)
Evaluating began
tensor(0.1751, grad_fn=<MeanBackward0>)
tensor(0.5306)
Evaluating wgan
tensor(-3.2745, grad_fn=<MeanBackward0>)
tensor(0.9342)
